In [ ]:
!pip install pymorphy3 # for python < 3.11 install pymorphy2
!pip install pymystem3
!pip install nltk
!pip install natasha

## При обработке текстов выделяют несколько этапов анализа.

1. Токенизация (графематический анализ) - выделение абзацев, предложений, токенов. Если абзацы в HTML выделяются довольно просто - по тегам <p>, то с выделением предложений и слов могут быть проблемы. ```Г. Мурманск был основан 3 апреля 1915 г. ниже впадения р. Туломы в Кольский залив. Минимальные IP-адреса: 109.124.97.0 - 109.124.97.3.```
2. Морфологический анализ (стемминг, лемматизация) - определение начальной формы слова или его псевдопрефикса, грамматических параметров. Подробнее описан ниже.
3. Синтаксический анализ - определение связей между словами (деревья зависимостей) или синтаксически связанных групп слов (деревья составляющих). Первые больше подходят для русского языка, вторые - для английского.

[Деревья зависимостей](https://en.wikipedia.org/wiki/Dependency_grammar)
![Деревья зависимостей](https://camo.githubusercontent.com/69c14a92ea941425c457f68d31c8c82d689cc0bdd03084834e1206304304b24a/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f302f30642f5765617265747279696e67746f756e6465727374616e64746865646966666572656e63655f253238322532392e6a7067)


[Деревья составляющих](https://en.wikipedia.org/wiki/Constituent_(linguistics))
![Деревья составляющих](https://camo.githubusercontent.com/bcc935da739466a3db252c38cb5eb57145c0da984212c5f7277848663848aac7/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f7468756d622f342f34392f436f6d706574696e675f73656e74656e63655f6469616772616d732e706e672f37353070782d436f6d706574696e675f73656e74656e63655f6469616772616d732e706e67)

- Семантический анализ - определение смысла слова и работа с ним (```за'мок``` vs ```замо'к```, удаление ребра связанного графа; не путать с ```он видел их семью своими глазами``` где имеет место грамматическая неоднозначность ```семья-семь```).

Задачей морфологического анализа является определение начальной формы слова, его части речи и грамматических параметров. В некоторых случаях от слова требуется только начальная форма, в других - только начальная форма и часть речи.

Существует два больших подхода к морфологическому анализу: **стемминг** и **поиск по словарю**. Для проведения стемминга оставляется справочник всех окончаний для данного языка. Для пришедшего слова проверяется его окончание и по нему делается прогноз начальной формы и части речи.
Например, мы создаем справочник, в котором записываем все окончания прилагательных: -ому, -ему, -ой, -ая, -ий, -ый, ... Теперь все слова, которые имеют такое окончание будут считаться прилагаельными: синий, циклический, красного, больному. Заодно прилагательными будут считаться причастия (делающий, строившему) и местоимения (мой, твой, твоему). Также не понятно что делать со словами, имеющими пустое окончание. Отдельную проблему составляют такие слова, как стекло, больной, вина, которые могут разбираться несколькими вариантами (это явление называется омонимией). Помимо этого, стеммер может просто откусывать окончания, оставляя лишь псевдооснову.

Большинство проблем здесь решается, но точность работы бессловарных стеммеров находится на уровне 80%. Чтобы повысить точность испольуют морфологический анализ со словарем. Разработчики составляют словарь слов, встретившихся в текстах (здесь можно найти пример такого словаря). Теперь каждое слово будет искаться в словаре и не предсказываться, а выдаваться точно. Для слов, отсутствующих в словаре, может применяться предсказание, пообное работе стеммера.

### Морфологический анализ
Есть несколько наиболее распространенных библиотеки для морфологического анализа текстов на Python: pymorphy2, pymystem и nltk. Рассмотрим работу с ними.

Библиотеки pymorphy основана на словаре [OpenCorpora](https://opencorpora.org/) и позволяет проводить анализ отдельных слов, то есть предварительно необходимо провести графематический анализ.

In [2]:
import pymorphy3
from pprint import pprint
morph=pymorphy3.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.
wordform=morph.parse('стекло')  # Проведем анализ слова "стекло"...
pprint(wordform)                 # ... и посмотрим на результат.

[Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=0.690476, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 0),)),
 Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='стекло', score=0.285714, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 3),)),
 Parse(word='стекло', tag=OpencorporaTag('VERB,perf,intr neut,sing,past,indc'), normal_form='стечь', score=0.023809, methods_stack=((DictionaryAnalyzer(), 'стекло', 1015, 3),))]


OpeccorporaTag - это объект, представляющий морфологическую информацию о слове в рамках русского языка в системе OpenCorpora. Этот объект содержит информацию о частях речи, роде, числе, падеже и других грамматических характеристиках слова.

Теперь, если мы анализируем слово "стекло" с новым списком морфологических разборов:

1. Существительное "стекло" в именительном падеже единственного числа, среднего рода. Вероятность этой формы составляет 69.05%.
2. Существительное "стекло" в винительном падеже единственного числа, среднего рода. Вероятность этой формы составляет 28.57%.
3. Глагол "стечь" в прошедшем времени единственного числа, среднего рода. Вероятность этой формы составляет 2.38%.

Снова применяя самый простой метод выбора начальной формы, мы можем сказать, что слово "стекло" скорее всего используется в форме существительного в именительном падеже, так как это наиболее вероятная форма с вероятностью около 69%. Этот подход обеспечивает примерно 90% точности при выборе начальной формы и до 80% точности при учете грамматических параметров.

In [3]:
wordform[0].score

0.690476

In [4]:
wordform[0].tag

OpencorporaTag('NOUN,inan,neut sing,nomn')

Описание OpencorporaTag('NOUN,inan,neut sing,nomn'):

1. NOUN: Это указывает на часть речи слова, в данном случае, существительное.
2. inan: Это указывает на неодушевленность существительного, что означает, что это не живое существо или неодушевленный объект.
3. neut: Это указывает на род существительного, в данном случае, средний род.
4. sing: Это указывает на число, в данном случае, единственное число (существительное имеет только одну форму).
5. nomn: Это указывает на падеж существительного, в данном случае, именительный падеж (когда существительное выполняет функцию подлежащего в предложении).

[Полный список](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html)

Pymorphy умеет синтезировать нужные нам формы слова. Для этого необходимо получить объект типа Parse для нужного слова, а затем вызвать функцию inflect.

In [5]:
wordform[0].inflect({'plur','datv'})

Parse(word='стёклам', tag=OpencorporaTag('NOUN,inan,neut plur,datv'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стёклам', 157, 8),))

In [6]:
for item in wordform[0].lexeme:
    print(item)

Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 0),))
Parse(word='стекла', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стекла', 157, 1),))
Parse(word='стеклу', tag=OpencorporaTag('NOUN,inan,neut sing,datv'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стеклу', 157, 2),))
Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 3),))
Parse(word='стеклом', tag=OpencorporaTag('NOUN,inan,neut sing,ablt'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стеклом', 157, 4),))
Parse(word='стекле', tag=OpencorporaTag('NOUN,inan,neut sing,loct'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стекле', 157, 5),))
Parse(word='стёкла', tag=Opencor

А ещё pymorphy умеет предсказывать незнакомые слова.

In [7]:
wordform=morph.parse('флексить') 
wordform

[Parse(word='флексить', tag=OpencorporaTag('INFN,perf,tran'), normal_form='флексить', score=1.0, methods_stack=((FakeDictionary(), 'флексить', 858, 0), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'ксить')))]

In [8]:
for item in wordform[0].lexeme:
    print(item.word, item.score, item.tag)

флексить 1.0 INFN,perf,tran
флексил 1.0 VERB,perf,tran masc,sing,past,indc
флексила 1.0 VERB,perf,tran femn,sing,past,indc
флексило 1.0 VERB,perf,tran neut,sing,past,indc
флексили 1.0 VERB,perf,tran plur,past,indc
флекшу 1.0 VERB,perf,tran sing,1per,futr,indc
флексим 1.0 VERB,perf,tran plur,1per,futr,indc
флексишь 1.0 VERB,perf,tran sing,2per,futr,indc
флексите 1.0 VERB,perf,tran plur,2per,futr,indc
флексит 1.0 VERB,perf,tran sing,3per,futr,indc
флексят 1.0 VERB,perf,tran plur,3per,futr,indc
флексим 1.0 VERB,perf,tran sing,impr,incl
флексимте 1.0 VERB,perf,tran plur,impr,incl
флекси 1.0 VERB,perf,tran sing,impr,excl
флексите 1.0 VERB,perf,tran plur,impr,excl
флексивший 1.0 PRTF,perf,tran,past,actv masc,sing,nomn
флексившего 1.0 PRTF,perf,tran,past,actv masc,sing,gent
флексившему 1.0 PRTF,perf,tran,past,actv masc,sing,datv
флексившего 1.0 PRTF,perf,tran,past,actv anim,masc,sing,accs
флексивший 1.0 PRTF,perf,tran,past,actv inan,masc,sing,accs
флексившим 1.0 PRTF,perf,tran,past,actv masc,

Вместо Pymorphy можно использовать PyMystem. Его плюсом является тот факт, что он сам проводит графематический анализ и снимает омонимию.

Функция lemmatize делит текст на слова и знаки препинания, а затем возвращает для них только начальную форму.

Функция analyze возвращает не только начальную форму, но и всю информацию о слове, как это делал перед этим Pymorphy.

Как видно из примера, делает он это не всегда корректно, но нам не придется думать о том, какое вариант разбора следует взять.

In [9]:
import pymystem3

In [10]:
mystem=pymystem3.Mystem()
pprint(mystem.lemmatize('эти типы стали есть в цеху.'))
pprint(mystem.analyze('эти типы стали есть в цеху.'))

['этот',
 ' ',
 'тип',
 ' ',
 'становиться',
 ' ',
 'есть',
 ' ',
 'в',
 ' ',
 'цех',
 '.',
 '\n']
[{'analysis': [{'gr': 'APRO=(им,мн|вин,мн,неод)', 'lex': 'этот', 'wt': 1}],
  'text': 'эти'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=(вин,мн|им,мн)',
                'lex': 'тип',
                'wt': 0.8700298642}],
  'text': 'типы'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,нп=прош,мн,изъяв,сов',
                'lex': 'становиться',
                'wt': 0.9821285244}],
  'text': 'стали'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,несов,пе=инф', 'lex': 'есть', 'wt': 0.0492236161}],
  'text': 'есть'},
 {'text': ' '},
 {'analysis': [{'gr': 'PR=', 'lex': 'в', 'wt': 0.9999917878}], 'text': 'в'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=(дат,ед|местн,ед)', 'lex': 'цех', 'wt': 1}],
  'text': 'цеху'},
 {'text': '.'},
 {'text': '\n'}]


То есть результатом является список токенов (в том числе и пробельных или знаков препинания), для части из которых имеется результат анализа, который хранится в словаре с ключём analysis. Анализ хранит в списке один или несколько вариантов разбора, у каждого из которых есть лемма lex, набор грамматических параметров gr и некоторый вес wt, который показывает степень уверенности системы в правильности ответа.

In [30]:
my_res=mystem.analyze('эти типы стали есть в цеху.')
if 'analysis' in my_res[0].keys(): # Проверяем, что это не разделитель.
    print(my_res[0]['analysis'][0]['gr'].split("=")[0]) # Берем из него анализ, из того грамматическсие параметы, 
                                                        # а из них выделяем часть речи.

APRO


In [11]:
import nltk # Иностранный морфологический анализатор.

Перед началом использования необходимо загрузить необходимые библиотеки или корпуса.

In [12]:
nltk.download() # По дороге будут появляться поле ввода. Грузит всё из Сети.

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [13]:
nltk.download(['averaged_perceptron_tagger_ru', 'stopwords', 'punkt']) # можно скачать сразу нужные пакеты

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /Users/roman/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/roman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/roman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Функция word_tokenize возвращает начальные формы слов.

Функция pos_tag возвращает список начальных форм и их частей речи.

In [14]:
tokens = nltk.word_tokenize('Эти типы стали есть в цеху') # Токенизация.
bi_tokens = list(nltk.bigrams(tokens))
pprint(tokens)
pprint(bi_tokens)

['Эти', 'типы', 'стали', 'есть', 'в', 'цеху']
[('Эти', 'типы'),
 ('типы', 'стали'),
 ('стали', 'есть'),
 ('есть', 'в'),
 ('в', 'цеху')]


In [15]:
pos = nltk.pos_tag(tokens) # Частеречная разметка.
bi_pos = list(nltk.bigrams(pos))
pos, bi_pos

([('Эти', 'JJ'),
  ('типы', 'NNP'),
  ('стали', 'NNP'),
  ('есть', 'NNP'),
  ('в', 'NNP'),
  ('цеху', 'NN')],
 [(('Эти', 'JJ'), ('типы', 'NNP')),
  (('типы', 'NNP'), ('стали', 'NNP')),
  (('стали', 'NNP'), ('есть', 'NNP')),
  (('есть', 'NNP'), ('в', 'NNP')),
  (('в', 'NNP'), ('цеху', 'NN'))])

У NLTK заведен список стоп-слов, которые лучше фильтровать при анализе текстов. Но их не очень много. Зато самые мешающиеся.

In [17]:
# Оставим только те слова, которых нет в списке стоп-слов.
filtered_words = [token for token in tokens if token not in nltk.corpus.stopwords.words('russian')]
print('всего русских стоп-слов', len(nltk.corpus.stopwords.words('russian')))
filtered_words

всего русских стоп-слов 151


['Эти', 'типы', 'стали', 'цеху']

In [19]:
# nltk.corpus.stopwords.words('russian')

Ту же самую задачу в других библиотеках можно решить при помощи фильтра частей речи. Можно считать, что значимыми являются лишь существительные, прилагательные, глаголы, причастия и деепричастия. Ниже приведены названия частей речи для разных библиотек

Существует простая в использовании библиотека Natasha, которая позволяет сделать все тоже самое очень простыми действиями

In [41]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [55]:
segmenter = Segmenter() # токенизация
morph_vocab = MorphVocab() # морфология

emb = NewsEmbedding() # построение эмбедингов
morph_tagger = NewsMorphTagger(emb) # разметка морфологии
syntax_parser = NewsSyntaxParser(emb) # разметка синтаксиса
ner_tagger = NewsNERTagger(emb) # разметка именованых сущностей

names_extractor = NamesExtractor(morph_vocab)

text = 'Г. Мурманск был основан 3 апреля 1915 г. ниже впадения р. Туломы в Кольский залив. Минимальные IP-адреса: 109.124.97.0 - 109.124.97.3.'
doc = Doc(text)

In [56]:
doc.segment(segmenter)

In [57]:
doc.tokens

[DocToken(stop=1, text='Г'),
 DocToken(start=1, stop=2, text='.'),
 DocToken(start=3, stop=11, text='Мурманск'),
 DocToken(start=12, stop=15, text='был'),
 DocToken(start=16, stop=23, text='основан'),
 DocToken(start=24, stop=25, text='3'),
 DocToken(start=26, stop=32, text='апреля'),
 DocToken(start=33, stop=37, text='1915'),
 DocToken(start=38, stop=39, text='г'),
 DocToken(start=39, stop=40, text='.'),
 DocToken(start=41, stop=45, text='ниже'),
 DocToken(start=46, stop=54, text='впадения'),
 DocToken(start=55, stop=56, text='р'),
 DocToken(start=56, stop=57, text='.'),
 DocToken(start=58, stop=64, text='Туломы'),
 DocToken(start=65, stop=66, text='в'),
 DocToken(start=67, stop=75, text='Кольский'),
 DocToken(start=76, stop=81, text='залив'),
 DocToken(start=81, stop=82, text='.'),
 DocToken(start=83, stop=94, text='Минимальные'),
 DocToken(start=95, stop=104, text='IP-адреса'),
 DocToken(start=104, stop=105, text=':'),
 DocToken(start=106, stop=118, text='109.124.97.0'),
 DocToken(s

In [58]:
doc.sents

[DocSent(stop=82, text='Г. Мурманск был основан 3 апреля 1915 г. ниже впа..., tokens=[...]),
 DocSent(start=83, stop=134, text='Минимальные IP-адреса: 109.124.97.0 - 109.124.97...., tokens=[...])]

In [59]:
doc.tag_morph(morph_tagger)

In [60]:
doc.sents[0].morph.print()

                   Г PROPN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing
                   . PUNCT
            Мурманск PROPN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
                 был AUX|Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
             основан VERB|Aspect=Perf|Gender=Masc|Number=Sing|Tense=Past|Variant=Short|VerbForm=Part|Voice=Pass
                   3 ADJ
              апреля NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                1915 ADJ
                   г NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                   . PUNCT
                ниже ADJ|Degree=Cmp
            впадения NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                   р NOUN|Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing
                   . PUNCT
              Туломы PROPN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing
                   в ADP
            Кольский ADJ|Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing
    

In [61]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)

In [62]:
{_.text: _.lemma for _ in doc.tokens}

{'Г': 'г',
 '.': '.',
 'Мурманск': 'мурманск',
 'был': 'быть',
 'основан': 'основать',
 '3': '3',
 'апреля': 'апрель',
 '1915': '1915',
 'г': 'г',
 'ниже': 'ниже',
 'впадения': 'впадение',
 'р': 'р',
 'Туломы': 'тулома',
 'в': 'в',
 'Кольский': 'кольский',
 'залив': 'залив',
 'Минимальные': 'минимальный',
 'IP-адреса': 'ip-адрес',
 ':': ':',
 '109.124.97.0': '109.124.97.0',
 '-': '-',
 '109.124.97.3': '109.124.97.3'}

In [63]:
doc.parse_syntax(syntax_parser)
doc.sents[0].syntax.print()

        ┌──► Г        nsubj:pass
        │    .        
        │    Мурманск 
        │ ┌► был      aux:pass
┌─┌─┌───└─└─ основан  
│ │ │ ┌─└►┌─ 3        obl
│ │ │ │   └► апреля   flat
│ │ │ │   ┌► 1915     amod
│ │ │ └──►└─ г        nmod
│ │ └──────► .        punct
│ │       ┌► ниже     case
│ │   ┌─┌►└─ впадения nmod
│ │   └►└─┌─ р        nmod
│ │       │  .        
│ │       └► Туломы   appos
│ │     ┌──► в        case
│ │     │ ┌► Кольский amod
│ └────►└─└─ залив    obl
└──────────► .        punct


In [64]:
doc.tag_ner(ner_tagger)
doc.ner.print()

Г. Мурманск был основан 3 апреля 1915 г. ниже впадения р. Туломы в 
   LOC─────                                                        
Кольский залив. Минимальные IP-адреса: 109.124.97.0 - 109.124.97.3.
LOC───────────                                                     
